This notebook implements the very beginning of the von Fintel and Heim lecture notes on intensional semantics.  Lots more to do here!

I'm providing it mainly as an example of how to use type s in the lambda notebook and how to add an intensionalization operation, but it really doesn't get past the first example.

In [ ]:
reload_lamb()

In [ ]:
type_s = types.BasicType("s")
lang.get_system().add_basic_type(type_s)
intensional_types = meta.get_type_system()
intensional_types

In [ ]:
%%lamb
# some standard extensional entries
||cat|| = L x_e: Cat_<e,t>(x)
||gray|| = L x_e: Gray_<e,t>(x)
||john|| = John_e
||julius|| = Julius_e
||inP|| = L x_e: L y: In(y, x)
||texas|| = Texas_e
||isV|| = L f_<e,t> : f
||dance|| = L x_e: Dance_<e,t>(x)
||see|| = L x_e : L y_e : See(y, x)
||Alfonso|| = Alfonso_e

In [ ]:
(inP * texas) * julius

How could an item like `must` be accounted for in this sort of extensional system?  Let's see what type inference gets us:

In [ ]:
%%lamb
||must|| = L x_? : Must_<?,t>(x)

In [ ]:
must_test_result = must * (julius * dance)
must_test_result

In [ ]:
must_test_result.tree(derivations=True)

In [ ]:
# Zoom in on what's happening in the final step
must_test_result[0].content.derivation

Type &lt;t,t&gt; is the only inferrable option.  This is not really satisfactory, as &lt;t,t&gt; describes a very small set.  In fact, there are just four functions:

 * 1 $\rightarrow$ 1, 0 $\rightarrow$ 0
 * 1 $\rightarrow$ 0, 0 $\rightarrow$ 1
 * 1 $\rightarrow$ 0, 0 $\rightarrow$ 0
 * 1 $\rightarrow$ 1, 0 $\rightarrow$ 1

Hence, some richer ("intensional") representation is needed.  Possible worlds are involved in one kind of analysis of intensionality.

In [ ]:
intensional_types

In [ ]:
# set up evaluation world as a parameter to the interpretation function
int_system = lang.get_system().copy()
eval_w = lang.te("w_s")
int_system.assign_controller = lang.AssignmentController(specials=[eval_w])
lang.set_system(int_system)

In [ ]:
%%lamb 
||dance|| = L x_e: Dance(w, x)
||Alfonso|| = Alfonso_e

In [ ]:
(Alfonso * dance)

In [ ]:
(Alfonso * dance).tree()

In [ ]:
def intension_fun(f, assignment=None):
    new_a = lang.Assignment(assignment)
    new_a.update({"w": lang.te("w_s")})
    result = meta.LFun("w_s", f.content.under_assignment(new_a))
    return lang.UnaryComposite(f, result, source="INT(%s)" % (f.name))

intension_fun(dance)

In [ ]:
intension_fun(julius)

In [ ]:
int_system.add_rule(lang.UnaryCompositionOp("INT", intension_fun))
int_system

In [ ]:
%%lamb
||a|| = L f_<e,t> : L g_<e,t> : Exists x_e : f(x) & g(x)

In [ ]:
%%lamb
||famous|| = L x_e : Famous(w,x)
||detective|| = L x_e : Detective(w,x)
||baker street 221B|| = BakerSt221B_e
||lives|| = L y_e : L x_e : LivesAt(w,x,y)

In [ ]:
simple = (a * detective) * (lives * baker_street_221B)
simple

In [ ]:
simple.results[0].tree(derivations=True)

In [ ]:
simple.compose()

In [ ]:
itw = lang.Item("In the world of S.H.", lang.te("L p_<s,t> : Forall w2_s : CompatibleWithSHStories(w_s, w2_s) >> p_<s,t>(w2_s)"))
itw

In [ ]:
%%lamb
||in the world of Sherlock Holmes|| = L p_<s,t> : Forall w2_s : CompatibleWithSHStories(w_s, w2_s) >> p_<s,t>(w2_s)

In [ ]:
# In the current setup, INT must be triggered explicitly by a unary compose step.
# Otherwise, it produces a composition failure.
in_the_world_of_Sherlock_Holmes * simple

In [ ]:
result = in_the_world_of_Sherlock_Holmes * (simple.compose())
result.reduce_all()
result

In [ ]:
result.results[0].tree(derivations=True)

Beginnings of attitude verbs, still needs work:

In [ ]:
%%lamb 
dox = Dox_<(e,s,s),t>
||believe|| = L p_<s,t> : L x_e : Forall w2_s : dox(x, w, w2) >> p(w2)

In [ ]:
lang.tp("<(e,s,s),t>")

In [ ]:
lang.te("f_<s,{<s,t>}>")

In [ ]:
%%lamb
dox = Dox_<(e,s),<s,t>>
||believe|| = L p_<s,t> : L x_e : Forall w2_s : dox(x, w)(w2) >> p(w2)

In [ ]:
%%lamb
dox = Dox_<(e,s),{<s,t>}>
bigcap = L a_{<s,t>} : Set w_s : Forall p_<s,t> : (p << a) & p(w)
||believe|| = L p_<s,t> : L x_e : Forall w2_s : w2 << bigcap(dox(x, w_s)) >> p(w2)

In [ ]:
believe.reduce_all() # can simplify this one slightly

In [ ]:
(believe * (simple * None))